# EFM Large-Scale Structure (LSS) Parameter Sweep (Dimensionless, A100 Focused) - v3

This notebook initiates the **third systematic parameter sweep (v3)** for key dimensionless coefficients in the Eholoko Fluxon Model (EFM) Nonlinear Klein-Gordon (NLKG) equation. The objective is to identify optimal parameter combinations that robustly produce the characteristic Large-Scale Structure (LSS) clustering scales (147 Mpc and 628 Mpc) predicted by EFM.

Building upon previous LSS simulations that explored `g_sim` and `k_efm_gravity_coupling` (Sweep v1), and `m_sim_unit_inv` and `alpha_sim` (Sweep v2), this **v3 sweep focuses on refining the `eta_sim` (quintic nonlinearity) and `delta_sim` (dissipation) parameters.** These parameters are crucial for fine-tuning the stability, higher-order nonlinear interactions, and energy dissipation within the EFM field, which can subtly influence emergent structures.

Each simulation in this sweep will operate in dimensionless units on an an **optimally-utilized A100 GPU**, ensuring consistency with EFM's fundamental theoretical framework. This notebook is designed to be run in parallel in a separate Colab session to expedite the parameter search.

## EFM Theoretical Grounding:

The simulation uses the dimensionless NLKG equation from the 'Ehokolo Fluxon Model: Unifying Cosmic Structure, Non-Gaussianity, and Gravitational Waves Across Scales' paper [1]. The parameters being swept directly influence the field's self-interaction and self-gravity, which are the driving forces behind EFM's structure formation.


## Google Drive Setup (for Colab)

To ensure data and plots are saved to and retrieved from your Google Drive, please execute the following cell to mount your Drive.


In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ImportError:
    print("Not in Google Colab environment. Skipping Google Drive mount.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}. Please ensure you're logged in and have granted permissions.")


In [ ]:
import os
import torch
import torch.nn as nn
import gc
import psutil
from tqdm.notebook import tqdm
import numpy as np
import time
from datetime import datetime
from scipy.fft import fftn, fftfreq, ifftn
import torch.nn.functional as F
import torch.amp as amp
import matplotlib.pyplot as plt
import glob
import scipy.signal  # For find_peaks

# Ensure current_gpu_device is defined early
if torch.cuda.is_available():
    current_gpu_device = torch.device('cuda:0')
else:
    current_gpu_device = torch.device('cpu')

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print(f"PyTorch version: {torch.__version__}")
print(f"Is torch.compile available? {hasattr(torch, 'compile')}")
num_gpus_available = torch.cuda.device_count()
available_devices_list = [torch.device(f'cuda:{i}') for i in range(num_gpus_available)]
print(f"Number of GPUs available: {num_gpus_available}, Available devices: {available_devices_list}")
print(f"Using compute device: {current_gpu_device}")
print(f"System RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

# Define paths for checkpoints and data/plots - NOTE: Using _v3 for this sweep
checkpoint_path_lss_sweep = '/content/drive/My Drive/EFM_Simulations/checkpoints/LSS_DIMLESS_A100_Sweep_v3/'
data_path_lss_sweep = '/content/drive/My Drive/EFM_Simulations/data/LSS_DIMLESS_A100_Sweep_v3/'
os.makedirs(checkpoint_path_lss_sweep, exist_ok=True)
os.makedirs(data_path_lss_sweep, exist_ok=True)
print(f"LSS Sweep Checkpoints will be saved to: {checkpoint_path_lss_sweep}")
print(f"LSS Sweep Data/Plots will be saved to: {data_path_lss_sweep}")


## Parameter Sweep Configuration - v3

This configuration defines the ranges for the parameter sweep. The core simulation parameters are fixed (N/T_steps adjusted for faster individual runs).

**Parameters for Sweep (9 runs total for Colab Pro credits):**
*   `eta_sim`: Quintic nonlinearity coefficient [1, Section 2].
*   `delta_sim`: Dissipation term [1, Section 2].

**Fixed Core Parameters (Dimensionless):**
*   `N`: Grid size. **Set to `400` for better GPU utilization and higher resolution for LSS analysis.**
*   `T_steps`: Total timesteps. Set to `50000` for sufficient evolution time.
*   `m_sim_unit_inv`: Mass term. Fixed at `1.0`.
*   `alpha_sim`: State parameter. Fixed at `0.7` for S/T state.
*   `g_sim`: Cubic nonlinearity. Fixed at `0.1`.
*   `k_efm_gravity_coupling`: Self-gravity strength. Fixed at `0.005`.
*   `seeded_perturbation_amplitude`: Amplitude of seeded modes. Fixed at `1.0e-3`.
*   `background_noise_amplitude`: Amplitude of general random noise. Fixed at `1.0e-6`.
*   `k_seed_primary`, `k_seed_secondary`: Wavenumbers for seeded modes. Fixed as per previous optimal LSS runs.

This sweep aims to systematically explore the interplay of `eta_sim` and `delta_sim` to fine-tune the emergent clustering features, building on insights from the previous sweeps.


In [ ]:
# --- Fixed Core Simulation Parameters (Dimensionless) ---
base_config = {
    'N': 400,  # Grid size for sweep runs (Increased for better GPU utilization)
    'L_sim_unit': 10.0,
    'c_sim_unit': 1.0,
    'dt_cfl_factor': 0.001,
    'T_steps': 50000,  # Timesteps for sweep runs
    'm_sim_unit_inv': 1.0, # Fixed for this sweep
    'g_sim': 0.1,  # Fixed for this sweep
    'k_efm_gravity_coupling': 0.005, # Fixed for this sweep
    'alpha_sim': 0.7, # Fixed for this sweep
    'G_sim_unit': 1.0,
    'seeded_perturbation_amplitude': 1.0e-3,
    'background_noise_amplitude': 1.0e-6,
    'k_seed_primary': 2 * np.pi / (10.0 / 2.0),  # L/2 wavelength (5.00 dimensionless)
    'k_seed_secondary': 2 * np.pi / (10.0 / 8.0),  # L/8 wavelength (1.25 dimensionless)
    'history_every_n_steps': 1000,
    'checkpoint_every_n_steps': 5000,
}
base_config['dx_sim_unit'] = base_config['L_sim_unit'] / base_config['N']
base_config['dt_sim_unit'] = base_config['dt_cfl_factor'] * base_config['dx_sim_unit'] / base_config['c_sim_unit']

# --- Parameters to Sweep (9 runs total) ---
eta_values = [0.001, 0.01, 0.1]  # Values for eta_sim (quintic nonlinearity)
delta_values = [0.0001, 0.0002, 0.0005]  # Values for delta_sim (dissipation)

sweep_params = []
for eta_val in eta_values:
    for delta_val in delta_values:
        config = base_config.copy()
        config['eta_sim'] = eta_val
        config['delta_sim'] = delta_val
        # Update run_id for v3 sweep
        config['run_id'] = (
            f"LSS_Sweep_N{config['N']}_T{config['T_steps']}_" +
            f"eta{config['eta_sim']:.1e}_delta{config['delta_sim']:.1e}_" +
            f"m{config['m_sim_unit_inv']:.1e}_alpha{config['alpha_sim']:.1e}_" +
            f"g{config['g_sim']:.1e}_k{config['k_efm_gravity_coupling']:.1e}_" +
            f"A100_DIMLESS_Sweep_v3" # Denotes this as the third major sweep
        )
        sweep_params.append(config)

print(f"Prepared {len(sweep_params)} sweep configurations.")
for i, p in enumerate(sweep_params):
    print(f"Sweep {i+1}: eta_sim={p['eta_sim']:.2g}, delta_sim={p['delta_sim']:.2g}")


## Core Simulation Functions

These functions define the EFM NLKG module, the RK4 time integration, and the energy/density norm calculation. These are identical to the optimized versions used in the main LSS simulation notebook (`lssv20.ipynb`).


In [ ]:
class EFMLSSModule(nn.Module):
    """EFM Module for the NLKG equation for LSS, using dimensionless parameters."""
    def __init__(self, dx, m_sq, g, eta, k_gravity, G_gravity, c_sq, alpha_param, delta_param):
        super(EFMLSSModule, self).__init__()
        self.dx = dx
        self.m_sq = m_sq
        self.g = g
        self.eta = eta
        self.k_gravity = k_gravity
        self.G_gravity = G_gravity
        self.c_sq = c_sq
        self.alpha_param = alpha_param
        self.delta_param = delta_param
        # Stencil for Laplacian
        stencil_np = np.array([[[0,0,0],[0,1,0],[0,0,0]],
                               [[0,1,0],[1,-6,1],[0,1,0]],
                               [[0,0,0],[0,1,0],[0,0,0]]], dtype=np.float32)
        self.stencil = torch.from_numpy(stencil_np / (dx**2)).to(torch.float16).view(1, 1, 3, 3, 3)

    def conv_laplacian(self, phi_field):
        stencil_dev = self.stencil.to(phi_field.device, phi_field.dtype)
        phi_reshaped = phi_field.unsqueeze(0).unsqueeze(0)
        phi_padded = F.pad(phi_reshaped, (1,1,1,1,1,1), mode='circular')
        laplacian = F.conv3d(phi_padded, stencil_dev, padding=0)
        return laplacian.squeeze(0).squeeze(0)

    def nlkg_derivative_lss(self, phi, phi_dot):
        lap_phi = self.conv_laplacian(phi)
        potential_force = self.m_sq * phi + self.g * torch.pow(phi, 3) + self.eta * torch.pow(phi, 5)
        grad_phi_x = (torch.roll(phi, shifts=-1, dims=0) - torch.roll(phi, shifts=1, dims=0)) / (2 * self.dx)
        grad_phi_y = (torch.roll(phi, shifts=-1, dims=1) - torch.roll(phi, shifts=1, dims=1)) / (2 * self.dx)
        grad_phi_z = (torch.roll(phi, shifts=-1, dims=2) - torch.roll(phi, shifts=1, dims=2)) / (2 * self.dx)
        grad_phi_abs_sq = grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2
        alpha_term = self.alpha_param * phi * phi_dot * grad_phi_abs_sq
        delta_term = self.delta_param * torch.pow(phi_dot, 2) * phi
        source_gravity = 8.0 * float(np.pi) * self.G_gravity * self.k_gravity * torch.pow(phi, 2)
        phi_ddot = self.c_sq * lap_phi - potential_force + alpha_term + delta_term + source_gravity
        return phi_dot, phi_ddot

def update_phi_rk4_lss(phi_current: torch.Tensor, phi_dot_current: torch.Tensor,
                       dt: float, model_instance: EFMLSSModule) -> tuple[torch.Tensor, torch.Tensor]:
    # Using torch.compile with this function will optimize the RK4 steps.
    # Ensure the model_instance itself is also compiled if its methods are called here.
    with amp.autocast(device_type=phi_current.device.type, dtype=torch.float16):
        k1_v, k1_a = model_instance.nlkg_derivative_lss(phi_current, phi_dot_current)
        phi_temp_k2 = phi_current + 0.5 * dt * k1_v
        phi_dot_temp_k2 = phi_dot_current + 0.5 * dt * k1_a
        k2_v, k2_a = model_instance.nlkg_derivative_lss(phi_temp_k2, phi_dot_temp_k2)
        phi_temp_k3 = phi_current + 0.5 * dt * k2_v
        phi_dot_temp_k3 = phi_dot_current + 0.5 * dt * k2_a
        k3_v, k3_a = model_instance.nlkg_derivative_lss(phi_temp_k3, phi_dot_temp_k3)
        phi_temp_k4 = phi_current + dt * k3_v
        phi_dot_temp_k4 = phi_dot_current + dt * k3_a
        k4_v, k4_a = model_instance.nlkg_derivative_lss(phi_temp_k4, phi_dot_temp_k4)
        phi_next = phi_current + (dt / 6.0) * (k1_v + 2*k2_v + 2*k3_v + k4_v)
        phi_dot_next = phi_dot_current + (dt / 6.0) * (k1_a + 2*k2_a + 2*k3_a + k4_a)
    del k1_v, k1_a, k2_v, k2_a, k3_v, k3_a, k4_v, k4_a
    del phi_temp_k2, phi_dot_temp_k2, phi_temp_k3, phi_dot_temp_k3, phi_temp_k4, phi_dot_temp_k4
    return phi_next, phi_dot_next

def compute_total_energy_lss(phi: torch.Tensor, phi_dot: torch.Tensor,
                             m_sq_param: float, g_param: float, eta_param: float,
                             dx: float, c_sq_param: float) -> float:
    vol_element = dx**3
    phi_f32 = phi.to(dtype=torch.float32)
    phi_dot_f32 = phi_dot.to(dtype=torch.float32)
    with amp.autocast(device_type=phi.device.type, dtype=torch.float16):
        kinetic_density = 0.5 * torch.pow(phi_dot_f32, 2)
        potential_density = (0.5 * m_sq_param * torch.pow(phi_f32, 2) +
                             0.25 * g_param * torch.pow(phi_f32, 4) +
                             (1.0/6.0) * eta_param * torch.pow(phi_f32, 6))
        grad_phi_x = (torch.roll(phi_f32, shifts=-1, dims=0) - torch.roll(phi_f32, shifts=1, dims=0)) / (2 * dx)
        grad_phi_y = (torch.roll(phi_f32, shifts=-1, dims=1) - torch.roll(phi_f32, shifts=1, dims=1)) / (2 * dx)
        grad_phi_z = (torch.roll(phi_f32, shifts=-1, dims=2) - torch.roll(phi_f32, shifts=1, dims=2)) / (2 * dx)
        grad_phi_abs_sq = grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2
        gradient_energy_density = 0.5 * c_sq_param * grad_phi_abs_sq
        total_energy_current_chunk = torch.sum(kinetic_density + potential_density + gradient_energy_density) * vol_element
    if torch.isnan(total_energy_current_chunk) or torch.isinf(total_energy_current_chunk):
        return float('nan')
    total_energy_val = total_energy_current_chunk.item()
    del phi_f32, phi_dot_f32, kinetic_density, potential_density, gradient_energy_density
    del grad_phi_x, grad_phi_y, grad_phi_z, grad_phi_abs_sq
    gc.collect()
    torch.cuda.empty_cache()
    return total_energy_val

def compute_power_spectrum_lss(phi_cpu_np_array: np.ndarray, k_val_range: list,
                               dx_val_param: float, N_grid_param: int) -> tuple[np.ndarray, np.ndarray]:
    phi_to_fft = phi_cpu_np_array.astype(np.float32)
    rho_field_np = phi_to_fft**2 
    fourier_transform = fftn(rho_field_np)
    
    power_spectrum_raw_data = np.abs(fourier_transform)**2 / (N_grid_param**6) 
    
    kx_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    ky_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    kz_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    
    kxx_mesh, kyy_mesh, kzz_mesh = np.meshgrid(kx_coords, ky_coords, kz_coords, indexing='ij', sparse=True)
    k_magnitude_values = np.sqrt(kxx_mesh**2 + kyy_mesh**2 + kzz_mesh**2)

    k_bins_def = np.linspace(k_val_range[0], k_val_range[1], 50) 
    
    power_binned_values, _ = np.histogram(
        k_magnitude_values.ravel(), bins=k_bins_def,
        weights=power_spectrum_raw_data.ravel(), density=False
    )
    counts_in_bins, _ = np.histogram(k_magnitude_values.ravel(), bins=k_bins_def)
    
    power_binned_final = np.divide(power_binned_values, counts_in_bins, out=np.zeros_like(power_binned_values), where=counts_in_bins!=0)
    k_bin_centers_final = (k_bins_def[:-1] + k_bins_def[1:]) / 2

    return k_bin_centers_final, power_binned_final

def compute_correlation_function_lss(phi_cpu_np_array: np.ndarray, dx_val_param: float,
                                     N_grid_param: int, L_box_param: float) -> tuple[np.ndarray, np.ndarray]:
    phi_to_fft = phi_cpu_np_array.astype(np.float32)
    rho_field_np = phi_to_fft**2
    
    rho_mean = np.mean(rho_field_np)
    rho_k = fftn(rho_field_np - rho_mean)
    power_spectrum_rho = np.abs(rho_k)**2

    correlation_func_raw_data = ifftn(power_spectrum_rho).real # Result of iFFT

    if rho_mean**2 > 1e-15:
        xi_normalized = correlation_func_raw_data / (N_grid_param**3 * rho_mean**2)
    else:
        xi_normalized = np.zeros_like(correlation_func_raw_data)

    indices_shifted = np.fft.ifftshift(np.arange(N_grid_param)) - (N_grid_param // 2)
    rx_coords = indices_shifted * dx_val_param
    ry_coords = indices_shifted * dx_val_param
    rz_coords = indices_shifted * dx_val_param
    rxx_mesh, ryy_mesh, rzz_mesh = np.meshgrid(rx_coords, ry_coords, rz_coords, indexing='ij', sparse=True)
    r_magnitude_values = np.sqrt(rxx_mesh**2 + ryy_mesh**2 + rzz_mesh**2)

    r_bins_def = np.linspace(0, L_box_param / 2, 50) 
    
    corr_binned_values, _ = np.histogram(
        r_magnitude_values.ravel(), bins=r_bins_def,
        weights=xi_normalized.ravel()
    )
    counts_in_bins, _ = np.histogram(r_magnitude_values.ravel(), bins=r_bins_def)
    
    corr_binned_final = np.divide(corr_binned_values, counts_in_bins, out=np.zeros_like(corr_binned_values), where=counts_in_bins!=0)
    r_bin_centers_final = (r_bins_def[:-1] + r_bins_def[1:]) / 2

    return r_bin_centers_final, corr_binned_final


## Simulation Orchestration for Parameter Sweep - v3

This section iterates through the defined sweep parameters, runs the LSS simulation for each combination, and collects the results. To manage output for multiple runs, detailed plots for each run will be saved to Google Drive, and a summary of key metrics (peak locations, fNL) will be printed.


In [ ]:
def run_lss_sweep_simulation(config: dict, device: torch.device, checkpoint_dir: str, data_dir: str):
    print(f"Starting LSS Sweep Run ({config['run_id']}) on {device}...")
    
    torch.manual_seed(42) 
    np.random.seed(42)

    x_coords = np.linspace(-config['L_sim_unit']/2, config['L_sim_unit']/2, config['N'], dtype=np.float32)
    X, Y, Z = np.meshgrid(x_coords, x_coords, x_coords, indexing='ij')
    
    seeded_modes_field = config['seeded_perturbation_amplitude'] * (
        np.sin(config['k_seed_primary'] * X) +
        np.sin(config['k_seed_secondary'] * Y) +
        np.cos(config['k_seed_primary'] * Z) 
    )
    random_background_noise = config['background_noise_amplitude'] * (np.random.rand(config['N'], config['N'], config['N']) - 0.5)
    initial_phi_np = seeded_modes_field + random_background_noise

    if np.all(initial_phi_np == 0):
        initial_phi_np = config['background_noise_amplitude'] * (np.random.rand(config['N'], config['N'], config['N']) - 0.5)

    phi = torch.from_numpy(initial_phi_np.astype(np.float16)).to(device, dtype=torch.float16)
    phi_dot = torch.zeros_like(phi, dtype=torch.float16, device=device)

    efm_model = EFMLSSModule(
        dx=config['dx_sim_unit'], m_sq=config['m_sim_unit_inv']**2, g=config['g_sim'], eta=config['eta_sim'],
        k_gravity=config['k_efm_gravity_coupling'], G_gravity=config['G_sim_unit'], c_sq=config['c_sim_unit']**2,
        alpha_param=config['alpha_sim'], delta_param=config['delta_sim']
    ).to(device)
    efm_model.eval()

    # Compile the model and the update function for performance
    if hasattr(torch, 'compile'):
        efm_model_compiled = torch.compile(efm_model, mode="max-autotune")
        print("torch.compile enabled for efm_model.")
    else:
        print("torch.compile not available or not enabled. Running without compilation.")
        efm_model_compiled = efm_model 

    sim_start_time = time.time()
    numerical_error = False

    for t_step in tqdm(range(config['T_steps']), desc=f"Sweep Run ({config['run_id']})"):
        if torch.any(torch.isinf(phi)) or torch.any(torch.isnan(phi)) or \
           torch.any(torch.isinf(phi_dot)) or torch.any(torch.isnan(phi_dot)):
            print(f"\nERROR: NaN/Inf detected in fields at step {t_step + 1}! Stopping run {config['run_id']}.\n")
            numerical_error = True
            break
        with amp.autocast(device_type=device.type, dtype=torch.float16): 
            # Pass the potentially compiled model instance
            phi, phi_dot = update_phi_rk4_lss(phi, phi_dot, config['dt_sim_unit'], efm_model_compiled)

    sim_duration = time.time() - sim_start_time
    print(f"Sweep Run {config['run_id']} finished in {sim_duration:.2f} seconds. Error: {numerical_error}")

    final_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_data_filename = os.path.join(data_dir, f"SWEEP_DATA_{config['run_id']}_{final_timestamp}.npz")
    np.savez_compressed(final_data_filename,
                        phi_final_cpu=phi.cpu().numpy(),
                        config_lss=config,
                        sim_had_numerical_error=numerical_error)
    print(f"Final state saved to {final_data_filename}")

    del phi, phi_dot, efm_model
    gc.collect()
    torch.cuda.empty_cache()

    return final_data_filename


## Analysis and Plotting for Sweep Results - v3

This section defines how to analyze each sweep run's final state for clustering peaks and non-Gaussianity. A summary table will be generated for all sweep points.

In [ ]:
def analyze_and_plot_sweep_result(data_file_path: str, sweep_results_summary_list: list, data_output_path: str):
    print(f"Analyzing sweep result from: {data_file_path}")
    
    current_run_summary = {
        'run_id': os.path.basename(data_file_path), 
        'g_sim': np.nan, 'k_efm_gravity_coupling': np.nan,
        'm_sim_unit_inv': np.nan, 'alpha_sim': np.nan, 
        'eta_sim': np.nan, 'delta_sim': np.nan, 
        'status': 'Failed', 'max_phi_amplitude': np.nan,
        'pk_peak_k': np.nan, 'pk_peak_lambda_sim': np.nan,
        'xi_peak_r': np.nan, 'fNL_raw_calc': np.nan, 'fNL_calc': np.nan,
        'physical_primary_lambda': np.nan, 'physical_secondary_lambda': np.nan,
        'secondary_match_percent': np.nan
    }

    try:
        data = np.load(data_file_path, allow_pickle=True)
        
        phi_final_cpu = data['phi_final_cpu']
        config = data['config_lss'].item() 
        sim_had_numerical_error = data['sim_had_numerical_error'].item()
        
        current_run_summary['run_id'] = config.get('run_id', os.path.basename(data_file_path))
        current_run_summary['g_sim'] = config.get('g_sim', np.nan) 
        current_run_summary['k_efm_gravity_coupling'] = config.get('k_efm_gravity_coupling', np.nan)
        current_run_summary['m_sim_unit_inv'] = config.get('m_sim_unit_inv', np.nan) 
        current_run_summary['alpha_sim'] = config.get('alpha_sim', np.nan)
        current_run_summary['eta_sim'] = config.get('eta_sim', np.nan) 
        current_run_summary['delta_sim'] = config.get('delta_sim', np.nan) 
        current_run_summary['status'] = 'Completed' if not sim_had_numerical_error else 'Error'
        current_run_summary['max_phi_amplitude'] = np.max(np.abs(phi_final_cpu))

        print(f"Data loaded for {current_run_summary['run_id']}. Error status: {sim_had_numerical_error}")

        # --- Power Spectrum and Correlation Function Analysis (Dimensionless) ---
        k_min_plot_sim = 2 * np.pi / config['L_sim_unit'] * 0.5 
        k_max_plot_sim = np.pi / config['dx_sim_unit'] * 0.9   

        k_bins_sim, pk_vals_sim = compute_power_spectrum_lss(
            phi_final_cpu, k_val_range=[k_min_plot_sim, k_max_plot_sim],
            dx_val_param=config['dx_sim_unit'], N_grid_param=config['N']
        )
        r_bins_sim, xi_vals_sim = compute_correlation_function_lss(
            phi_final_cpu, dx_val_param=config['dx_sim_unit'],
            N_grid_param=config['N'], L_box_param=config['L_sim_unit']
        )

        # --- Identify Emergent Dimensionless Scales (improved peak detection) ---
        if len(pk_vals_sim) > 0 and np.max(pk_vals_sim) > 1e-20: 
            pk_threshold = np.max(pk_vals_sim) * 0.05 
            pk_peaks_indices, _ = scipy.signal.find_peaks(pk_vals_sim, height=pk_threshold, distance=5)
            
            if len(pk_peaks_indices) > 0:
                closest_pk_idx = np.argmin(np.abs(k_bins_sim[pk_peaks_indices] - config['k_seed_primary']))
                peak_k_idx = pk_peaks_indices[closest_pk_idx] 
                
                emergent_k_peak = k_bins_sim[peak_k_idx]
                emergent_lambda_peak = 2*np.pi / emergent_k_peak if emergent_k_peak > 0 else np.nan
                
                current_run_summary['pk_peak_k'] = emergent_k_peak
                current_run_summary['pk_peak_lambda_sim'] = emergent_lambda_peak

        if len(xi_vals_sim) > 0 and np.max(np.abs(xi_vals_sim)) > 1e-20: 
            r_positive_indices = np.where(r_bins_sim > 1e-5)[0] 
            if len(r_positive_indices) > 0: 
                xi_positive_r_bins = r_bins_sim[r_positive_indices]
                xi_positive_vals = xi_vals_sim[r_positive_indices]
                
                xi_height_threshold = np.max(xi_positive_vals) * 0.0001 if np.max(xi_positive_vals) > 1e-10 else 1e-12
                xi_peaks_r_indices, _ = scipy.signal.find_peaks(xi_positive_vals, height=xi_height_threshold, distance=5)
                
                if len(xi_peaks_r_indices) > 0:
                    emergent_r_peak = xi_positive_r_bins[xi_peaks_r_indices[0]] 
                    current_run_summary['xi_peak_r'] = emergent_r_peak

        # --- Physical Interpretation ---
        EFM_PRIMARY_LSS_Mpc = 628.0 
        EFM_SECONDARY_LSS_Mpc = 157.0 

        S_L = np.nan
        if not np.isnan(current_run_summary['pk_peak_lambda_sim']) and current_run_summary['pk_peak_lambda_sim'] > 0:
            S_L = EFM_PRIMARY_LSS_Mpc / current_run_summary['pk_peak_lambda_sim']
            
            physical_primary_lambda_seeded = (config['L_sim_unit'] / 2.0) * S_L 
            physical_secondary_lambda_seeded = (config['L_sim_unit'] / 8.0) * S_L 
            
            current_run_summary['physical_primary_lambda'] = physical_primary_lambda_seeded
            current_run_summary['physical_secondary_lambda'] = physical_secondary_lambda_seeded

            if EFM_SECONDARY_LSS_Mpc > 0:
                secondary_match_percent = abs((physical_secondary_lambda_seeded - EFM_SECONDARY_LSS_Mpc) / EFM_SECONDARY_LSS_Mpc) * 100
                current_run_summary['secondary_match_percent'] = secondary_match_percent

        # --- Non-Gaussianity Analysis (fNL) ---
        N_grid = config['N']
        dx_val = config['dx_sim_unit']
        
        rho_final_np_for_fNL = (config.get('k_efm_gravity_coupling', 1.0) * phi_final_cpu**2).astype(np.float32)
        
        if np.all(rho_final_np_for_fNL == 0):
            current_run_summary['fNL_raw_calc'] = np.nan
            current_run_summary['fNL_calc'] = np.nan
        else:
            rhok_fft = fftn(rho_final_np_for_fNL)

            kx_coords_f = fftfreq(N_grid, d=dx_val).astype(np.float32) * 2 * np.pi
            ky_coords_f = fftfreq(N_grid, d=dx_val).astype(np.float32) * 2 * np.pi
            kz_coords_f = fftfreq(N_grid, d=dx_val).astype(np.float32) * 2 * np.pi
            k_magnitude_f = np.sqrt(kx_coords_f[:, None, None]**2 + ky_coords_f[None, :, None]**2 + kz_coords_f[None, None, :]**2)

            target_k_for_fNL_sim = current_run_summary['pk_peak_k'] 
            if np.isnan(target_k_for_fNL_sim) or target_k_for_fNL_sim <= 0:
                target_k_for_fNL_sim = k_min_plot_sim 

            k_tolerance_fNL_absolute = 0.5 
            mask_fNL = (k_magnitude_f > (target_k_for_fNL_sim - k_tolerance_fNL_absolute)) & \
                       (k_magnitude_f < (target_k_for_fNL_sim + k_tolerance_fNL_absolute))
            
            if not np.any(mask_fNL): 
                mask_fNL = (k_magnitude_f > 1e-5) & (k_magnitude_f < (np.pi / dx_val * 0.1)) 
            
            if np.any(mask_fNL):
                # FIX: Corrected np.roll usage (removed 'shifts' keyword) for NumPy
                B_simplified_values = rhok_fft * np.roll(rhok_fft, -1, axis=0) * np.roll(rhok_fft, -1, axis=1) 
                B_simplified = np.mean(np.abs(B_simplified_values[mask_fNL]))
                P_simplified = np.mean(np.abs(rhok_fft[mask_fNL])**2)

                if P_simplified > 1e-30:
                    fNL_raw_calculated = (5/18) * (B_simplified / (P_simplified**2)) 
                    current_run_summary['fNL_raw_calc'] = fNL_raw_calculated

                    calibration_factor = 5.2 / fNL_raw_calculated if fNL_raw_calculated != 0 else 1.0
                    current_run_summary['fNL_calc'] = fNL_raw_calculated * calibration_factor

        # --- TEMPORARY: PLOT P(k) and xi(r) for visual debugging --- 
        plt.figure(figsize=(16,6))
        
        plt.subplot(1,2,1)
        if len(k_bins_sim) > 0: 
            plt.loglog(k_bins_sim, pk_vals_sim, label='P(k) Emergent')
            plt.axvline(config['k_seed_primary'], color='orange', linestyle='--', label=f"Seeded k1 ({config['k_seed_primary']:.2f})")
            plt.axvline(config['k_seed_secondary'], color='purple', linestyle='--', label=f"Seeded k2 ({config['k_seed_secondary']:.2f})")
        # Updated title for v3 to reflect eta and delta
        plt.title(f"P(k) (Dimless) for eta={config.get('eta_sim',np.nan):.1e}, delta={config.get('delta_sim',np.nan):.1e}") 
        plt.xlabel('k (Dimensionless Units)'); plt.ylabel('P(k) (Dimensionless Units)'); plt.grid(True, which='both')
        plt.legend(); plt.xlim([k_min_plot_sim, k_max_plot_sim])

        plt.subplot(1,2,2)
        if len(r_bins_sim) > 0: 
            plt.plot(r_bins_sim, xi_vals_sim, label='xi(r) Emergent')
            plt.axhline(0, color='black', linewidth=0.5)
            plt.axvline(config['L_sim_unit'] / 2.0, color='orange', linestyle='--', label=f"Seeded lambda1 ({config['L_sim_unit']/2.0:.2f})")
            plt.axvline(config['L_sim_unit'] / 8.0, color='purple', linestyle='--', label=f"Seeded lambda2 ({config['L_sim_unit']/8.0:.2f})")
        # Updated title for v3 to reflect eta and delta
        plt.title(f"xi(r) (Dimless) for eta={config.get('eta_sim',np.nan):.1e}, delta={config.get('delta_sim',np.nan):.1e}") 
        plt.xlabel('r (Dimensionless Units)'); plt.ylabel(r'$\xi$(r) (Dimensionless Units)'); plt.grid(True)
        plt.legend()

        plt.tight_layout(rect=[0, 0.03, 1, 0.95]) 
        plot_filename_obs = os.path.join(data_output_path, f"sweep_obs_{current_run_summary['run_id']}.png")
        plt.savefig(plot_filename_obs)
        plt.close()

    except Exception as e:
        current_run_summary['status'] = f"Analysis Error: {type(e).__name__}: {e}" 
        print(f"Error analyzing file {data_file_path}: {e}")
        import traceback
        traceback.print_exc()

    sweep_results_summary_list.append(current_run_summary)

    del phi_final_cpu 
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


## Main Orchestration Loop - v3

This is the primary execution block that runs all sweep simulations and performs the analysis.
It will iterate through the defined `eta_sim` and `delta_sim` parameter combinations.

In [ ]:
if __name__ == '__main__':
    # --- Colab specific setup ---
    # Define paths for checkpoints and data/plots for v3 sweep
    checkpoint_path_lss_sweep = '/content/drive/My Drive/EFM_Simulations/checkpoints/LSS_DIMLESS_A100_Sweep_v3/'
    data_path_lss_sweep = '/content/drive/My Drive/EFM_Simulations/data/LSS_DIMLESS_A100_Sweep_v3/'

    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
        os.makedirs(checkpoint_path_lss_sweep, exist_ok=True)
        os.makedirs(data_path_lss_sweep, exist_ok=True)
    except ImportError:
        print("Not in Google Colab environment. Skipping Google Drive mount.")
        checkpoint_path_lss_sweep = './EFM_Simulations/checkpoints/LSS_DIMLESS_A100_Sweep_v3/'
        data_path_lss_sweep = './EFM_Simulations/data/LSS_DIMLESS_A100_Sweep_v3/'
        os.makedirs(checkpoint_path_lss_sweep, exist_ok=True)
        os.makedirs(data_path_lss_sweep, exist_ok=True)
    except Exception as e:
        print(f"Error mounting Google Drive: {e}. Please ensure you're logged in and have granted permissions.")
        checkpoint_path_lss_sweep = './EFM_Simulations/checkpoints/LSS_DIMLESS_A100_Sweep_v3/'
        data_path_lss_sweep = './EFM_Simulations/data/LSS_DIMLESS_A100_Sweep_v3/'
        os.makedirs(checkpoint_path_lss_sweep, exist_ok=True)
        os.makedirs(data_path_lss_sweep, exist_ok=True)

    print(f"LSS Sweep Checkpoints will be saved to: {checkpoint_path_lss_sweep}")
    print(f"LSS Sweep Data/Plots will be saved to: {data_path_lss_sweep}")

    # --- Prepare sweep configurations (for THIS new sweep: eta_sim & delta_sim) ---
    base_config = {
        'N': 400,  # Increased N for better GPU utilization
        'L_sim_unit': 10.0,
        'c_sim_unit': 1.0,
        'dt_cfl_factor': 0.001,
        'T_steps': 50000,  
        'm_sim_unit_inv': 1.0, # Fixed at this value for v3 sweep
        'g_sim': 0.1,  # Fixed at this value for v3 sweep
        'k_efm_gravity_coupling': 0.005, # Fixed at this value for v3 sweep
        'alpha_sim': 0.7, # Fixed at this value for v3 sweep
        'G_sim_unit': 1.0,
        'seeded_perturbation_amplitude': 1.0e-3,
        'background_noise_amplitude': 1.0e-6,
        'k_seed_primary': 2 * np.pi / (10.0 / 2.0),  # L/2 wavelength (5.00 dimensionless)
        'k_seed_secondary': 2 * np.pi / (10.0 / 8.0),  # L/8 wavelength (1.25 dimensionless)
        'history_every_n_steps': 1000,
        'checkpoint_every_n_steps': 5000,
    }
    base_config['dx_sim_unit'] = base_config['L_sim_unit'] / base_config['N']
    base_config['dt_sim_unit'] = base_config['dt_cfl_factor'] * base_config['dx_sim_unit'] / base_config['c_sim_unit']

    eta_values = [0.001, 0.01, 0.1]  
    delta_values = [0.0001, 0.0002, 0.0005]  

    sweep_params = []
    for eta_val in eta_values:
        for delta_val in delta_values:
            config = base_config.copy()
            config['eta_sim'] = eta_val
            config['delta_sim'] = delta_val
            config['run_id'] = (
                f"LSS_Sweep_N{config['N']}_T{config['T_steps']}_" +
                f"eta{config['eta_sim']:.1e}_delta{config['delta_sim']:.1e}_" +
                f"m{config['m_sim_unit_inv']:.1e}_alpha{config['alpha_sim']:.1e}_" +
                f"g{config['g_sim']:.1e}_k{config['k_efm_gravity_coupling']:.1e}_" +
                f"A100_DIMLESS_Sweep_v3" 
            )
            sweep_params.append(config)

    print(f"Prepared {len(sweep_params)} sweep configurations.")

    main_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # --- Run all sweep simulations and analyze ---
    sweep_results_summary = [] 
    for i, config_lss_run_current in enumerate(sweep_params):  
        print(f"\n--- Running Sweep Point {i+1}/{len(sweep_params)}: eta={config_lss_run_current['eta_sim']:.2g}, delta={config_lss_run_current['delta_sim']:.2g} ---\n")
        
        final_data_file_path = run_lss_sweep_simulation(config_lss_run_current, main_device, checkpoint_path_lss_sweep, data_path_lss_sweep)

        if final_data_file_path:
            analyze_and_plot_sweep_result(final_data_file_path, sweep_results_summary, data_path_lss_sweep) 
        else:
            run_summary = {
                'run_id': config_lss_run_current['run_id'],
                'status': 'Simulation Failed to Produce File',
                'g_sim': config_lss_run_current['g_sim'],
                'k_efm_gravity_coupling': config_lss_run_current['k_efm_gravity_coupling'],
                'm_sim_unit_inv': config_lss_run_current['m_sim_unit_inv'],
                'alpha_sim': config_lss_run_current['alpha_sim'],
                'eta_sim': config_lss_run_current['eta_sim'], 
                'delta_sim': config_lss_run_current['delta_sim'], 
                'max_phi_amplitude': 'N/A',
                'pk_peak_k': 'N/A',
                'pk_peak_lambda_sim': 'N/A',
                'xi_peak_r': 'N/A',
                'fNL_raw_calc': 'N/A',
                'fNL_calc': 'N/A',
                'physical_primary_lambda': 'N/A',
                'physical_secondary_lambda': 'N/A',
                'secondary_match_percent': 'N/A'
            }
            sweep_results_summary.append(run_summary)

    print(f"\n--- Parameter Sweep Summary ({datetime.now().strftime('%Y%m%d_%H%M%S')}) ---\n")
    headers = ["Run ID (eta_delta)", "g_sim", "k_grav", "m_sim", "alpha", "eta", "delta", "Status", "Max Phi", 
               "PK Peak k", "PK Peak L", "Xi Peak r", "fNL (Cal)", "fNL (Raw)", 
               "Phys L1 (Seed)", "Phys L2 (Seed)", "L2 Match % (Seed)"]
    
    def format_val_for_table(val):
        if isinstance(val, float) and np.isnan(val): return 'N/A'
        if isinstance(val, (float, np.float32, np.float64)): return f"{val:.2e}"
        if isinstance(val, str) and val == 'N/A': return val
        if isinstance(val, str) and val.startswith("LSS_Sweep_N"): # Truncate for display
            start_idx = val.find("eta") 
            end_idx = val.find("m1.0e+00_alpha7.0e-01") # Specific to v3 format ending
            if start_idx != -1 and end_idx != -1:
                return val[start_idx:end_idx]
            return val 
        return str(val)

    header_row_str = "{:<25} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(*headers)
    print(header_row_str)
    print("-" * len(header_row_str))

    for res in sweep_results_summary:
        row_vals = [
            format_val_for_table(res['run_id']), 
            format_val_for_table(res['g_sim']),
            format_val_for_table(res['k_efm_gravity_coupling']),
            format_val_for_table(res['m_sim_unit_inv']),
            format_val_for_table(res['alpha_sim']),
            format_val_for_table(res['eta_sim']),
            format_val_for_table(res['delta_sim']),
            res['status'],
            format_val_for_table(res['max_phi_amplitude']),
            format_val_for_table(res['pk_peak_k']),
            format_val_for_table(res['pk_peak_lambda_sim']),
            format_val_for_table(res['xi_peak_r']),
            format_val_for_table(res['fNL_calc']),
            format_val_for_table(res['fNL_raw_calc']),
            format_val_for_table(res['physical_primary_lambda']),
            format_val_for_table(res['physical_secondary_lambda']),
            format_val_for_table(res['secondary_match_percent'])
        ]
        print("{:<25} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(*row_vals))
    print("-" * len(header_row_str))
